In [1]:
from tdc.single_pred import Tox
import deepchem
import xgboost
import rdkit
from rdkit import Chem
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem.EState import EState

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [12]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)
ecfpFeat = deepchem.deepchem.feat.CircularFingerprint(radius=4)
convMolFeat = deepchem.deepchem.feat.ConvMolFeaturizer()
#rdkitFeat = deepchem.deepchem.feat.CircularFingerprint(size=2048, radius=4)
xgb_reg_ecfp = xgboost.XGBRegressor()
xgb_reg_estate = xgboost.XGBRegressor()
#akReg = ak.StructuredDataRegressor(overwrite=True, max_trials=50, metrics=['mae'])

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('LD50_Zhu') 
    
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)  

    #trainMol = train.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    #validMol = valid.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    testMol = test.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))
    train_valMol = train_val.iloc[:,1].map(lambda x: Chem.MolFromSmiles(x))

    #featurize training, valid, and test data for xgboost
    #trainFeat = np.stack(np.array(trainMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    #validFeat = np.stack(np.array(validMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    testFeat = np.stack(np.array(testMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))
    train_valFeat = np.stack(np.array(train_valMol.map(lambda x: Fingerprinter.FingerprintMol(x)[1])))

    #featurize training, valid, and test data for xgboost
    #ecfp_f_train = ecfpFeat.featurize(train.iloc[:,1].to_list())
    #ecfp_f_valid = ecfpFeat.featurize(valid.iloc[:,1].to_list())
    ecfp_f_test = ecfpFeat.featurize(test.iloc[:,1].to_list())
    train_val_f = ecfpFeat.featurize(train_val.iloc[:,1].to_list())

    #featurize training, valid, and test data for the GCN
    cv_f_train = convMolFeat.featurize(train.iloc[:,1].to_list())
    cv_f_valid = convMolFeat.featurize(valid.iloc[:,1].to_list())
    cv_f_test = convMolFeat.featurize(test.iloc[:,1].to_list())

    #featurize datasets for rdkit descriptors
    #rdkit_f_train_val = pd.DataFrame(rdkitFeat.featurize(train_val.iloc[:,1].to_list()))
    #rdkit_f_test = pd.DataFrame(rdkitFeat.featurize(test.iloc[:,1].to_list()))

    #fit structured data regressor
    #akReg.fit(x=rdkit_f_train_val, y=train_val.iloc[:,2])

    #convert training and validation data into a deepchem dataset for the gcn
    gcn_train_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_train, y=np.array(train.iloc[:,2]), ids=np.array(train.iloc[:,1].to_list()))
    gcn_valid_data = deepchem.deepchem.data.NumpyDataset(X=cv_f_valid, y=np.array(valid.iloc[:,2]), ids=np.array(valid.iloc[:,1].to_list()))

    #fit data on GCN
    reg = deepchem.deepchem.models.GraphConvModel(
        n_tasks=1, 
        dropout=.0005,
        dense_layer_size=1063,
        graph_conv_layers=[128, 128, 128],
        mode="regression",)
    callback = deepchem.deepchem.models.ValidationCallback(gcn_valid_data, 1000, metric)
    reg.fit(gcn_train_data, nb_epoch=100, callbacks=callback)

    #predict values on gcn and reshape array
    gcn_pred = reg.predict_on_batch(X=np.array(cv_f_test)).reshape(1478,)

    #fit xgboost model and store np ndarray in xgb_pred
    xgb_reg_estate.fit(X=train_valFeat, y=train_val.iloc[:,2], eval_metric="mae", verbose=True)
    xgb_reg_ecfp.fit(X=train_val_f, y=train_val.iloc[:,2], eval_metric="mae")
    pred_estate = xgb_reg_estate.predict(X=testFeat)
    pred_ecfp = xgb_reg_ecfp.predict(X=ecfp_f_test)
    
    #pred_ak = akReg.predict(rdkit_f_test)

    # store test predictions in y_pred_test
    y_pred_test = np.mean([ pred_estate, pred_ecfp, gcn_pred ], axis=0)

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 5907/5907 [00:02<00:00, 2057.61it/s]


Step 1000 validation: mean_absolute_error=0.578711
Step 2000 validation: mean_absolute_error=0.564238
Step 3000 validation: mean_absolute_error=0.570522
Step 4000 validation: mean_absolute_error=0.560579
Step 5000 validation: mean_absolute_error=0.562462


generating training, validation splits...


predictions for run #
1
[2.0024517 2.6602182 3.3649118 3.9714787 4.183733 ]


100%|██████████| 5907/5907 [00:02<00:00, 2846.88it/s]


...

Step 1000 validation: mean_absolute_error=0.585828
Step 2000 validation: mean_absolute_error=0.573847
Step 3000 validation: mean_absolute_error=0.554584
Step 4000 validation: mean_absolute_error=0.545601
Step 5000 validation: mean_absolute_error=0.528875


generating training, validation splits...


predictions for run #
4
[2.0955093 2.3321826 3.1685574 3.715656  4.1162286]


100%|██████████| 5907/5907 [00:01<00:00, 4161.32it/s]


Step 1000 validation: mean_absolute_error=0.57304
Step 2000 validation: mean_absolute_error=0.550126
Step 3000 validation: mean_absolute_error=0.526647
Step 4000 validation: mean_absolute_error=0.508091
predictions for run #
5
[1.9931126 2.604688  3.3086777 3.936863  4.158606 ]
Prediction List:
[{'ld50_zhu': array([2.0024517, 2.6602182, 3.3649118, ..., 2.180651 , 2.0403793,
       2.184006 ], dtype=float32)}, {'ld50_zhu': array([1.8227962, 2.549201 , 3.2872546, ..., 2.1752853, 1.9567846,
       2.2744467], dtype=float32)}, {'ld50_zhu': array([1.9957209, 2.3490694, 3.3056872, ..., 2.2099812, 2.2676606,
       2.3312263], dtype=float32)}, {'ld50_zhu': array([2.0955093, 2.3321826, 3.1685574, ..., 2.0684412, 1.957521 ,
       2.0807416], dtype=float32)}, {'ld50_zhu': array([1.9931126, 2.604688 , 3.3086777, ..., 2.3482597, 2.1783981,
       2.3829255], dtype=float32)}]


In [13]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

{'ld50_zhu': {'mae': 0.61}}
{'ld50_zhu': {'mae': 0.629}}
{'ld50_zhu': {'mae': 0.62}}
{'ld50_zhu': {'mae': 0.622}}
{'ld50_zhu': {'mae': 0.606}}


{'ld50_zhu': [0.617, 0.008]}